In [32]:
# Celda 1: Importar las funciones necesarias
# Asegúrate de que la ruta de importación sea correcta desde donde ejecutas el notebook.
# Si ejecutas el notebook desde 'tu_proyecto/', la importación será:
from app.classifier import limpiar_texto, clasificar_texto_por_reglas, limpiar_texto_factura, clasificar_factura_examenes
import re # Si necesitas re para algún ejemplo de texto


In [2]:
# Celda 2: Textos de ejemplo (similares a los que obtendrías del OCR)
texto_autorizacion = """
AUTORIZACION DE SERVICIOS Pagina 1 de 1
olicitada el: 05/03/2025 N° Solicitud 1
Autorizada el 05/03/2025 N° Autorización (POS) 248-87404861
Esta autorización es netamente administrativa.
"""

texto_orden_medica_palabras = """
PLAN DE MANEJO:
Paciente requiere consulta especializada por Urología.
Diagnóstico: N40X Hiperplasia de la próstata
Se solicita FORMULACIÓN de Tamsulosina.
Cita de control en 3 meses.
"""

texto_orden_medica_codigos = """
Observaciones médicas: Paciente refiere dolor lumbar.
Se solicita radiografía de columna. Código Z01.6
Remisión a fisioterapia.
"""

texto_orden_medica_mixto = """
Fecha: 2023-10-27
Paciente: Juan Perez
Diagnóstico: I10 - Hipertensión esencial (primaria)
Plan de Manejo:
1. Losartan 50mg cada día.
2. Dieta baja en sodio.
3. Cita de control en 1 mes.
"""

texto_soporte_recibido = """
FORMATO DE RECIBIDO USUARIO
Yo, abajo firmante, certifico que recibí a satisfacción los medicamentos formulados.
"""

texto_sin_clasificacion_clara = """
Hoja de evolución paciente.
Paciente refiere mejoría de los síntomas. Signos vitales estables.
Se continúa igual manejo.
"""

texto_vacio = ""

In [3]:
# Celda 4: Probar la función clasificar_texto_por_reglas
print("--- Clasificación por Reglas ---")
casos_de_prueba = {
    "Autorización": texto_autorizacion,
    "Orden Médica (Palabras)": texto_orden_medica_palabras,
    "Orden Médica (Códigos)": texto_orden_medica_codigos,
    "Orden Médica (Mixto)": texto_orden_medica_mixto,
    "Soporte Recibido": texto_soporte_recibido,
    "Sin Clasificación Clara": texto_sin_clasificacion_clara,
    "Texto Vacío": texto_vacio
}

for nombre_caso, texto_ejemplo in casos_de_prueba.items():
    clasificacion, texto_para_llm = clasificar_texto_por_reglas(texto_ejemplo)
    print(f"Caso: {nombre_caso}")
    print(f"  Texto Original (primeros 50 chars): {texto_ejemplo[:50].replace('\n', ' ')}...")
    print(f"  Clasificación por Reglas: {clasificacion}")
    # print(f"  Texto para LLM (si aplica): {texto_para_llm[:50].replace('\n', ' ')}...") # Puedes descomentar para ver
    print("-" * 10)

--- Clasificación por Reglas ---
Caso: Autorización
  Texto Original (primeros 50 chars):  AUTORIZACION DE SERVICIOS Pagina 1 de 1 olicitada...
  Clasificación por Reglas: autorizacion
----------
Caso: Orden Médica (Palabras)
  Texto Original (primeros 50 chars):  PLAN DE MANEJO: Paciente requiere consulta especi...
  Clasificación por Reglas: orden_medica
----------
Caso: Orden Médica (Códigos)
  Texto Original (primeros 50 chars):  Observaciones médicas: Paciente refiere dolor lum...
  Clasificación por Reglas: orden_medica
----------
Caso: Orden Médica (Mixto)
  Texto Original (primeros 50 chars):  Fecha: 2023-10-27 Paciente: Juan Perez Diagnóstic...
  Clasificación por Reglas: orden_medica
----------
Caso: Soporte Recibido
  Texto Original (primeros 50 chars):  FORMATO DE RECIBIDO USUARIO Yo, abajo firmante, c...
  Clasificación por Reglas: soporte_de_recibido
----------
Caso: Sin Clasificación Clara
  Texto Original (primeros 50 chars):  Hoja de evolución paciente. Paciente refier

In [35]:
# Textos de ejemplo para clasificar facturas

texto_1 = """
CUPS NOMBRE CANT VR UNIT AJUSTE VR PAC VR ENT
PROCEDIMIENTOS DIAGNOSTICO
873412 873412 
RADIOGRAFIA DE CADERA COMPARATIVAMédico :
1129509300 - OSCAR MIGUEL ARIZA RIVERA 
1,00 124.433,00 $ 0,00 $ 4.700,00 $ 119.733,00 $
"""

texto_2 = """
CUPS NOMBRE CANT VR UNIT AJUSTE VR PAC VR ENT
PROCEDIMIENTOS DIAGNOSTICO
19522 903426 
HEMOGLOBINA GLICOSILADAMédico : 20796133 -
GLORIA IMELDA LUQUE MELO 
1,00 76.200,00 $ 0,00 $ 0,00 $ 76.200,00 $
19722 903026 
MICROALBUMINURIAMédico : 20796133 - GLORIA
IMELDA LUQUE MELO 
1,00 62.900,00 $ 0,00 $ 0,00 $ 62.900,00 $
19729 902101
MONO TEST (PRUEBA DE LATEX PARA
MONONUCLEOSIS INFECCIOSAMédico : 20796133 -
GLORIA IMELDA LUQUE MELO
1,00 36.100,00 $ 0,00 $ 0,00 $ 36.100,00 $
19792 903859 
POTASIOMédico : 20796133 - GLORIA IMELDA LUQUE
MELO 
1,00 49.200,00 $ 0,00 $ 0,00 $ 49.200,00 $
19891 903864 
SODIOMédico : 20796133 - GLORIA IMELDA LUQUE
MELO 
1,00 40.700,00 $ 0,00 $ 0,00 $ 40.700,00 $
"""

texto_3 = """
CUPS NOMBRE CANT VR UNIT AJUSTE VR PAC VR ENT
PROCEDIMIENTOS DIAGNOSTICO
31112TI 881141
ECOGRAFIA DE TIROIDES CON TRANSDUCTOR DE 7
MHZ O MASMédico : 19405784 - PENAGOS JAIMES
OMAR
1,00 166.100,00 $ 0,00 $ 4.700,00 $ 161.400,00 $
"""

texto_4 = """
CUPS NOMBRE CANT VR UNIT AJUSTE VR PAC VR ENT
PROCEDIMIENTOS DIAGNOSTICO
21101MA 873210 
RADIOGRAFIA DE MANOMédico : 1073600084 -
NELSON ENRIQUE PUERTAS POVEDA 
1,00 70.000,00 $ 0,00 $ 4.700,00 $ 65.300,00 $
"""

texto_5 = """
CUPS NOMBRE CANT VR UNIT AJUSTE VR PAC VR ENT
PROCEDIMIENTOS DIAGNOSTICO
19140 906610
ANTIGENO ESPECIFICO PARA CANCER DE
PROSTATAMédico : 20796133 - GLORIA IMELDA
LUQUE MELO
1,00 214.800,00 $ 0,00 $ 4.700,00 $ 210.100,00 $
19237 903815 
COLESTEROL HDLMédico : 20796133 - GLORIA
IMELDA LUQUE MELO 
1,00 34.900,00 $ 0,00 $ 0,00 $ 34.900,00 $
19241 903816 
COLESTEROL LDLMédico : 20796133 - GLORIA
IMELDA LUQUE MELO 
1,00 41.100,00 $ 0,00 $ 0,00 $ 41.100,00 $
19242 903818 
COLESTEROL TOTALMédico : 20796133 - GLORIA
IMELDA LUQUE MELO 
1,00 42.300,00 $ 0,00 $ 0,00 $ 42.300,00 $
19775 907106 
PARCIAL DE ORINA INCLUIDO SEDIMENTOMédico :
20796133 - GLORIA IMELDA LUQUE MELO 
1,00 22.000,00 $ 0,00 $ 0,00 $ 22.000,00 $
19868 907009 
SANGRE OCULTA EN MFMédico : 20796133 - GLORIA
IMELDA LUQUE MELO 
1,00 10.400,00 $ 0,00 $ 0,00 $ 10.400,00 $
19940 903868 
TRIGLICERIDOSMédico : 20796133 - GLORIA IMELDA
LUQUE MELO 
1,00 23.200,00 $ 0,00 $ 0,00 $ 23.200,00 $
"""

texto_6 = """
CUPS NOMBRE CANT VR UNIT AJUSTE VR PAC VR ENT
CONSULTAS
39143OT 890282
CONSULTA DE PRIMERA VEZ POR ESPECIALISTA EN
OTORRINOLARINGOLOGIAMédico : 51602887 - ANA
ADELA TIBADUIZA HERRERA
1,00 71.900,00 $ 0,00 $ 4.700,00 $ 67.200,00 $
"""
texto_7 = """
CUPS NOMBRE CANT VR UNIT AJUSTE VR PAC VR ENT
CONSULTAS
39143MIC 890366
CONSULTA DE CONTROL O DE SEGUIMIENTO POR
ESPECIALISTA EN MEDICINA INTERNAMédico :
85449194 - MANUEL SALVADOR PEÑA CASTRO
1,00 71.900,00 $ 0,00 $ 4.700,00 $ 67.200,00 $
"""


texto_vacio = ""


casos_clasificacion_facturas = [
    {"nombre": "Factura con Examenes 1 ", "texto": texto_1, "esperado_examenesFacturados": 1},
    {"nombre": "Factura con Examenes 2 ", "texto": texto_2, "esperado_examenesFacturados": 1},
    {"nombre": "Factura con Examenes 3 ", "texto": texto_3, "esperado_examenesFacturados": 1},
    {"nombre": "Factura con Examenes 4 ", "texto": texto_4, "esperado_examenesFacturados": 1},
    {"nombre": "Factura con Examenes 5 ", "texto": texto_5, "esperado_examenesFacturados": 1},
    {"nombre": "Factura con Consulta 1 ", "texto": texto_6, "esperado_examenesFacturados": 0},
    {"nombre": "Factura con Consulta 2 ", "texto": texto_7, "esperado_examenesFacturados": 0},

]

print(f"\n{len(casos_clasificacion_facturas)} casos de prueba para clasificación de facturas definidos.")


7 casos de prueba para clasificación de facturas definidos.


In [36]:
# Celda 5: Pruebas para la Clasificación de Facturas (Exámenes)

print("\n=== PRUEBAS DE CLASIFICACIÓN DE FACTURAS (EXÁMENES) ===")
for caso in casos_clasificacion_facturas:
    nombre_caso = caso["nombre"]
    texto_ejemplo = caso["texto"]
    esperado_dict = {"examenesFacturados": caso["esperado_examenesFacturados"]} # Compararemos el valor
    
    print(f"\n--- Probando Caso: {nombre_caso} ---")
    print(f"Texto Original (primeros 50 chars): '{texto_ejemplo[:50].replace('\n', ' ')}...'")
    
    # Llamar directamente a la función de lógica
    resultado_obtenido_dict = clasificar_factura_examenes(texto_ejemplo)
    examenes_facturados_obtenido = resultado_obtenido_dict.get("examenesFacturados")
    decision_source_obtenido = resultado_obtenido_dict.get("decision_source", "N/A")
    
    print(f"  Resultado Obtenido: {resultado_obtenido_dict}")
    
    if examenes_facturados_obtenido == caso["esperado_examenesFacturados"]:
        print(f"  ✅ PRUEBA PASADA: Esperado '{caso['esperado_examenesFacturados']}', Obtenido '{examenes_facturados_obtenido}' (Fuente: {decision_source_obtenido})")
    else:
        print(f"  ❌ PRUEBA FALLIDA: Esperado '{caso['esperado_examenesFacturados']}', Obtenido '{examenes_facturados_obtenido}' (Fuente: {decision_source_obtenido})")

print("\n=== FIN DE LAS PRUEBAS DIRECTAS DE LÓGICA ===")


=== PRUEBAS DE CLASIFICACIÓN DE FACTURAS (EXÁMENES) ===

--- Probando Caso: Factura con Examenes 1  ---
Texto Original (primeros 50 chars): ' CUPS NOMBRE CANT VR UNIT AJUSTE VR PAC VR ENT PRO...'
  Resultado Obtenido: {'examenesFacturados': 1, 'decision_source': 'rules'}
  ✅ PRUEBA PASADA: Esperado '1', Obtenido '1' (Fuente: rules)

--- Probando Caso: Factura con Examenes 2  ---
Texto Original (primeros 50 chars): ' CUPS NOMBRE CANT VR UNIT AJUSTE VR PAC VR ENT PRO...'
  Resultado Obtenido: {'examenesFacturados': 1, 'decision_source': 'rules'}
  ✅ PRUEBA PASADA: Esperado '1', Obtenido '1' (Fuente: rules)

--- Probando Caso: Factura con Examenes 3  ---
Texto Original (primeros 50 chars): ' CUPS NOMBRE CANT VR UNIT AJUSTE VR PAC VR ENT PRO...'
  Resultado Obtenido: {'examenesFacturados': 1, 'decision_source': 'rules'}
  ✅ PRUEBA PASADA: Esperado '1', Obtenido '1' (Fuente: rules)

--- Probando Caso: Factura con Examenes 4  ---
Texto Original (primeros 50 chars): ' CUPS NOMBRE CANT VR UNI

In [24]:
clasificar_factura_examenes(texto_3)

{'examenesFacturados': 1, 'decision_source': 'rules'}

In [19]:
clasificar_factura_examenes(texto_3)

{'examenesFacturados': 1, 'decision_source': 'rules'}